# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"¶
## Задание 1.


## Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.  

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:  

KEYWORDS = ['python', 'парсинг']  

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>  

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
KEYWORDS = ['python', 'парсинг','программировать','архитектура','технологий','платформа']

In [3]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
# извлекаем статьи
articles = soup.find_all('article')

In [5]:
ArticlesData = pd.DataFrame(columns = {'Date','Title','Link'})

In [6]:
def get_data(title_element, article_time):
    
    title_text = title_element.text
    article_link = title_element.attrs.get('href')
    
    if 'сегодня' in article_time.split(' '):
        article_date = datetime.date.today()
    elif 'вчера' in article_time.split(' '):
        yesterday = datetime.datetime.now() - datetime.timedelta(1)
        article_date = yesterday.date()
    else:
        article_date = article_time
    
    return title_text, article_link, article_date
    
    

In [7]:
for artcl in articles:
    
    title_element = artcl.find('a', class_='post__title_link')
    artcl_element_0 = artcl.find('div', class_='post__text_v1')
    artcl_element_1 = artcl.find('div', class_='post__text_v2')

    for keyword in KEYWORDS:
        if title_element:
            if keyword in title_element.text:
                title_text, article_link, article_date = get_data(title_element,artcl.find('span', class_='post__time').text)
                article_data_sample = {'Date':article_date,'Title':title_text,'Link':article_link}
                ArticlesData = ArticlesData.append(article_data_sample, ignore_index=True)
                
            elif artcl_element_0:
                if keyword in artcl_element_0.text:
                    title_text, article_link, article_date = get_data(title_element,artcl.find('span', class_='post__time').text)
                    article_data_sample = {'Date':article_date,'Title':title_text,'Link':article_link}
                    ArticlesData = ArticlesData.append(article_data_sample, ignore_index=True)
                    
            elif artcl_element_1:
                if keyword in artcl_element_1.text:
                    title_text, article_link, article_date = get_data(title_element,artcl.find('span', class_='post__time').text)
                    article_data_sample = {'Date':article_date,'Title':title_text,'Link':article_link}
                    ArticlesData = ArticlesData.append(article_data_sample, ignore_index=True)
                    

In [8]:
ArticlesData

,Title,Date,Link
0,Создаем Booking приложение с Webix UI,2021-04-12,https://habr.com/ru/post/551988/


# Задание 2.

## Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]  

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>  

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.  
    

In [57]:
import json

In [58]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [59]:
NEWSFEED_REQUEST = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [60]:
params = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0'
    }

In [61]:
LeakageData = pd.DataFrame(columns = {'email','date','source','description'})

In [62]:
for email in EMAIL:
    res = requests.post(NEWSFEED_REQUEST, json={'emailAddresses': [email]},headers = params)
    extracted_data = json.loads(res.text) 
    if extracted_data:
        for key in extracted_data['breaches'].keys():
            description = extracted_data['breaches'][key]['description']
            date = extracted_data['breaches'][key]['publishDate']
            source = extracted_data['breaches'][key]['site']
            site = extracted_data['breaches'][key]['site']
            data_sample = {'email':email,'date':date,'source':source,'description':description}
            LeakageData = LeakageData.append(data_sample,ignore_index=True)
    

In [63]:
LeakageData

,description,source,date,email
0,"In July and August 2016, two criminals execute...",parapa.mail.ru,2017-02-14T00:00:00Z,xxx@x.ru
1,Popular Russian social networking platform VKo...,vk.com,2016-10-29T00:00:00Z,xxx@x.ru
2,"In October of 2013, criminals penetrated Adobe...",adobe.com,2016-10-21T00:00:00Z,xxx@x.ru
3,"In July and August of 2016, two criminals carr...",cfire.mail.ru,2017-02-14T00:00:00Z,xxx@x.ru
4,"In March 2016, CDProjektRed.com.com's forum da...",cdprojektred.com,2017-01-31T00:00:00Z,xxx@x.ru
5,"In June 2016, a cache of over 51 million user ...",imesh.com,2016-10-23T00:00:00Z,xxx@x.ru
6,"At an unconfirmed date, online Arizona newspap...",azcentral.com,2020-01-03T00:00:00Z,yyy@y.com
7,"In January 2020, the online poll website Wishb...",wishbone.io,2020-05-28T00:00:00Z,yyy@y.com
8,"In October 2017, a customer database belonging...",myheritage.com,2017-11-04T00:00:00Z,yyy@y.com
9,"At an unconfirmed date, the Russian-language m...",forums.vkmonline.com,2021-02-11T00:00:00Z,yyy@y.com
